# HealthAIBot

In [2]:
# export env variables before running this block
import os

class KEYS:
    OPENAI_API_KEY = os.getenv("OPENAI_API_KEY", "your-openai-api-key")
    TAVILY_API_KEY = os.getenv("TAVILY_API_KEY", "your-tavily-api-key")

In [1]:
# /usr/bin/env python3
# healthAiBot/healthaibot/utils/utils.py
"""
Utility functions for HealthBot operations.
"""

from pydantic import BaseModel, Field
from typing import List, Optional, Any

from langchain_openai import ChatOpenAI
from langchain_ollama.chat_models import ChatOllama


class HealthBotState(BaseModel):
    # Allow either raw dicts or LangChain BaseMessage objects
    messages: List[Any] = Field(
        default_factory=list,
        description="List of conversation messages (dicts or LangChain messages) including tool calls for traceability"
    )
    topic: Optional[str] = None
    focus: Optional[str] = None
    search_results: Optional[str] = None
    summary: Optional[str] = None
    question: Optional[str] = None
    quiz_question: Optional[str] = None
    quiz_answer: Optional[str] = None
    quiz_grade: Optional[str] = None
    grading: Optional[str] = None
    continue_flag: Optional[str] = None
    previous_questions: List[str] = Field(default_factory=list)
    tool_call_events: List[Any] = Field(
        default_factory=list, 
        description="Legacy tool call tracking - use messages for better traceability"
    )
    llm: Optional[Any] = None


class HealthBotUtils:
    """
    Utility functions for HealthBot operations.
    """
    def __init__(
        self,
        llm_type: str,
        model_name: str,
        temperature: float = 0.7,
    ) -> None:
        """
        Initialize the HealthBotUtils class.
        Parameters:
            llm_type: Type of LLM to use ('openai' or 'ollama').
            model_name: Name of the model to use.
            temperature: Sampling temperature for the LLM.
        """
        self.llm_type = llm_type
        self.model_name = model_name
        self.temperature = temperature

    def get_llm(
        self,
    ) -> ChatOpenAI | ChatOllama:
        """
        Get the LLM instance based on the specified type.
        Returns:
            An instance of ChatOpenAI or ChatOllama.
        """
        if self.llm_type == "openai":
            return ChatOpenAI(
                model=self.model_name,
                temperature=self.temperature
            )
        elif self.llm_type == "ollama":
            return ChatOllama(
                model=self.model_name,
                temperature=self.temperature
            )
        else:
            raise ValueError("Unsupported LLM type. Choose 'openai' or 'ollama'.")

    def reset_state(
        self,
        llm: ChatOpenAI | ChatOllama,
    ) -> HealthBotState:
        """
        Reset the state of the HealthBot.
        """
        # Clear previous health information to maintain privacy
        return HealthBotState(llm=llm)

    def parse_quiz(
        self,
        quiz_text: str,
    ) -> tuple[str, list[str]]:
        """
        Parse the quiz text into a question and options.
        Now handles both single questions and multiple choice questions.
        """
        # Simple parser to split question and options
        lines = quiz_text.strip().split('\n')
        question = ""
        options = []
        
        for line in lines:
            # Check for multiple choice options
            if line.startswith("a)") or line.startswith("b)") or line.startswith("c)") or line.startswith("d)"):
                options.append(line)
            elif line.lower().startswith("question:"):
                question = line[len("Question:"):].strip()
            elif line and not line.startswith("summary:") and not line.startswith("previous questions:"):
                if question:
                    question += " " + line.strip()
                else:
                    question = line.strip()
        
        # If no options found, this is a single question (not MCQ)
        # Return empty options list to indicate open-ended question
        return question.strip(), options


In [3]:
"""Utility functions for HealthBot agent operations with quiz flow enforcement."""

from datetime import datetime
import os
from langchain_tavily import TavilySearch
from langchain.tools import tool


@tool("tavily_search_tool", return_direct=True)
def tavily_search_tool(topic: str) -> str:
    """Search authoritative medical sources (NIH, Mayo Clinic, WebMD) for the given topic."""
    if not os.environ.get("TAVILY_API_KEY"):
        raise ValueError("Missing Tavily API key. Please export TAVILY_API_KEY before running the agent.")
    search = TavilySearch()
    query = f"{topic} site:nih.gov OR site:mayoclinic.org OR site:webmd.com"
    return str(search.invoke(query))


class GraphHelper:
    def __init__(self) -> None:  # No special init needed
        pass

    # ---------------- Core Interaction Nodes -----------------
    def ask_patient(self, state: HealthBotState) -> HealthBotState:
        try:
            topic = input("What health topic or medical condition would you like to learn about? ")
        except EOFError:
            print("\nInput ended unexpectedly. Exiting HealthBot.")
            exit(0)
        state.topic = topic
        print(f"You have chosen to learn about: {state.topic}")
        state.messages.append({
            "role": "user",
            "content": f"I want to learn about {state.topic}.",
            "action": "topic_selection",
            "timestamp": str(datetime.now())
        })
        return state

    def generate_assistant_message(self, state: HealthBotState) -> HealthBotState:
        # Seed conversation and include an assistant message that simulates a tool call request.
        state.messages = [
            {"role": "system", "content": "You are a helpful medical information assistant."},
            {"role": "user", "content": f"I want to learn about {state.topic}."},
            {"role": "assistant", "content": f"Initiating search for {state.topic} using tavily_search_tool.", "tool_calls": [
                {"id": "call_tavily_1", "name": "tavily_search_tool", "arguments": {"topic": state.topic}}
            ]}
        ]
        return state

    def ask_for_focus(self, state: HealthBotState) -> HealthBotState:
        if not state.focus:
            try:
                focus = input("Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter: ")
            except EOFError:
                print("\nInput ended unexpectedly. Using no specific focus.")
                focus = ""
            if focus.strip():
                state.focus = focus.strip()
            state.messages.append({
                "role": "user",
                "content": f"Focus selection: {state.focus if state.focus else 'No specific focus'}",
                "action": "focus_selection",
                "timestamp": str(datetime.now())
            })
        return state

    def search_tavily(self, state: HealthBotState) -> HealthBotState:
        state.tool_call_events.append({
            "event": "tool_call", "tool": "tavily_search_tool", "topic": state.topic
        })
        return state

    def summarize_results(self, state: HealthBotState) -> HealthBotState:
        llm = state.llm
        focus = getattr(state, 'focus', None)
        # If no real search results (API key missing or fallback placeholder), avoid hallucination
        if not state.search_results or 'Missing Tavily API key' in state.search_results:
            state.summary = (
                "Search unavailable because Tavily API key is missing. "
                "Set TAVILY_API_KEY and restart to generate an evidence-based summary."
            )
            return state
        base_prompt = (
            "You are a medical information assistant. Summarize the search results for a patient.\n\n"
            "MANDATORY FORMAT & RULES (FOLLOW EXACTLY):\n"
            "1. Output MUST be EXACTLY 3 TO 4 paragraphs. No other number is acceptable.\n"
            "2. Paragraphs are separated by ONE blank line (a single empty line).\n"
            "3. Each paragraph MUST be between 3 and 5 sentences (inclusive).\n"
            "4. Use ONLY information present in the search results. If something isn't there, do NOT invent it.\n"
            "5. If an expected aspect is missing, explicitly state: 'The search results do not provide information about <missing aspect>'.\n"
            "6. Do NOT include bullet lists, numbering, headings, markdown, or metadata. Plain paragraphs only.\n"
            "7. If you cannot satisfy ALL rules with given content, write EXACTLY this sentence alone: 'The search results are insufficient to produce a compliant summary.'\n"
            "8. Do NOT mention these instructions or justify your formatting.\n\n"
            "QUALITY GUIDELINES:\n"
            "- Use clear, patient-friendly language.\n"
            "- Avoid redundancy; group related facts.\n"
            "- Prefer concrete facts over vague generalities.\n\n"
            "ACCEPTABLE EXAMPLE (3 paragraphs):\n"
            "Paragraph 1: Overview sentences 1-5.\n"
            "\nParagraph 2: Focused detail sentences 1-4.\n"
            "\nParagraph 3: Limitations + missing info sentences 1-3.\n\n"
            "UNACCEPTABLE EXAMPLES (DO NOT DO):\n"
            "- A single long block (fails rule 1).\n"
            "- 5 paragraphs (fails rule 1).\n"
            "- Paragraphs with 1–2 sentences (fails rule 3).\n"
            "- Bullet lists or headings (fails rule 6).\n\n"
        )
        if focus:
            base_prompt += f"FOCUS REQUIREMENT: Emphasize information about '{focus}'.\n\n"
        base_prompt += (
            "FORMAT: Write EXACTLY 3 TO 4 paragraphs separated by blank lines. Do not include headers, bullet points, or numbered lists.\n\n"
            "SEARCH RESULTS TO SUMMARIZE:\n"
        )
        prompt = base_prompt + (state.search_results or "")
        state.messages.append({
            "role": "user", "content": f"Requesting summary generation for topic: {state.topic}",
            "action": "summarize_results", "focus": focus if focus else "None"
        })
        if llm is None:
            state.summary = "LLM not initialized."
            return state
        summary = llm.invoke(prompt)
        state.summary = summary.content if hasattr(summary, 'content') else str(summary)
        state.messages.append({
            "role": "assistant",
            "content": f"Generated summary for {state.topic} ({len(state.summary)} characters)",
            "action": "summarize_results_complete",
            "summary_length": str(len(state.summary))
        })
        return state

    def present_summary(self, state: HealthBotState) -> HealthBotState:
        print("\nHere is a summary of what you asked about:\n")
        print(state.summary)
        return state

    def comprehension_prompt(self, state: HealthBotState) -> HealthBotState:
        try:
            input("\nPress Enter when you are ready to take a comprehension check.")
        except EOFError:
            print("\nInput ended unexpectedly. Proceeding with comprehension check.")
        return state

    # ---------------- Quiz Flow Nodes -----------------
    def create_quiz(self, state: HealthBotState) -> HealthBotState:
        llm = state.llm
        previous = list(getattr(state, 'previous_questions', []))
        prompt = (
            "Create ONE comprehension question based EXCLUSIVELY on the provided summary below.\n\n"
            "STRICT REQUIREMENTS:\n"
            "1. Create ONLY ONE question (open-ended)\n"
            "2. The question must be answerable ONLY using information from the summary\n"
            "3. No outside knowledge\n"
            "4. Test key understanding of the summary\n"
            "5. Do NOT reveal the answer\n"
            "6. Do NOT repeat any previous questions\n\n"
            "FORMAT: Output just the question text.\n\n"
            f"SUMMARY:\n{state.summary}\n\n"
            f"PREVIOUS QUESTIONS:\n{previous if previous else 'None'}\n\n"
            "QUESTION:"
        )
        state.messages.append({
            "role": "user", "content": f"Requesting quiz question for {state.topic}",
            "action": "create_quiz", "previous_questions_count": str(len(previous))
        })
        if llm is None:
            state.quiz_question = "LLM not initialized to create quiz question."
            return state
        raw = llm.invoke(prompt)
        raw_text = raw.content.strip() if hasattr(raw, 'content') else str(raw).strip()
        candidate_lines = [ln.strip() for ln in raw_text.split('\n') if ln.strip()]
        selected = ""
        for ln in candidate_lines:
            if '?' in ln and not selected:
                selected = ln
            elif '?' in ln and selected:
                state.messages.append({
                    "role": "assistant", "content": f"Discarded extra question: {ln[:80]}",
                    "action": "create_quiz_sanitizer"
                })
        if not selected and candidate_lines:
            selected = candidate_lines[0]
        for prefix in ["question:", "q:", "q1:"]:
            if selected.lower().startswith(prefix):
                selected = selected[len(prefix):].strip()
        if not selected.endswith('?'):
            selected = selected.rstrip('.') + '?'
        if selected not in previous:
            previous.append(selected)
        else:
            state.messages.append({
                "role": "assistant", "content": "Duplicate question detected (kept).",
                "action": "create_quiz_duplicate"
            })
        state.previous_questions = previous
        state.quiz_question = selected
        preview = selected[:100] + "..." if len(selected) > 100 else selected
        state.messages.append({
            "role": "assistant", "content": f"Generated sanitized quiz question for {state.topic}",
            "action": "create_quiz_complete", "question_preview": preview
        })
        return state

    def present_quiz(self, state: HealthBotState) -> HealthBotState:
        print("\nQuiz Question:\n")
        print(state.quiz_question)
        return state

    def get_quiz_answer(self, state: HealthBotState) -> HealthBotState:
        try:
            answer = input("\nEnter your answer to the quiz question: ")
        except EOFError:
            print("\nInput ended unexpectedly. Exiting HealthBot.")
            exit(0)
        state.quiz_answer = answer
        state.messages.append({
            "role": "user", "content": f"Quiz answer: {answer}",
            "action": "quiz_answer_submission", "question": state.quiz_question,
            "timestamp": str(datetime.now())
        })
        return state

    def grade_quiz(self, state: HealthBotState) -> HealthBotState:
        llm = state.llm
        prompt = (
            "You are a strict grading assistant. You must grade the user's answer using ONLY the provided SUMMARY.\n"
            "If the answer invents information not present in the SUMMARY, penalize it.\n"
            "If the answer contradicts the SUMMARY, penalize it.\n"
            "If the answer partially matches, give a middle grade.\n"
            "If the answer fully and accurately reflects key points in the SUMMARY, give a high grade.\n\n"
            "RESTRICTIONS:\n"
            "- You SHOULD NOT use any knowledge outside the SUMMARY.\n"
            "- Do NOT add new facts.\n"
            "- Justification MUST cite only facts/phrases that appear in the SUMMARY.\n\n"
            "ALLOWED GRADES:\nA = Completely accurate based only on SUMMARY\nB = Mostly accurate, minor omissions\nC = Partially accurate, missing important points\nD = Limited accuracy, several errors or omissions\nF = Incorrect or largely not based on SUMMARY\n\n"
            "OUTPUT FORMAT (must follow exactly, no extra lines):\n"
            "Grade: <A|B|C|D|F>\nJustification: <one concise sentence using only SUMMARY info>\n\n"
            f"SUMMARY (sole source of truth):\n{state.summary}\n\n"
            f"QUESTION:\n{state.quiz_question}\n\n"
            f"USER ANSWER:\n{state.quiz_answer}\n\n"
            "Now produce ONLY the required two-line format."
        )
        state.messages.append({
            "role": "user", "content": f"Requesting grade for quiz on {state.topic}",
            "action": "grade_quiz", "user_answer": state.quiz_answer or ""
        })
        if llm is None:
            state.grading = "LLM not initialized to grade quiz answer."
            return state
        grading = llm.invoke(prompt)
        raw_text = grading.content if hasattr(grading, 'content') else str(grading)

        # Post-process to enforce exact format.
        lines = [line.strip() for line in raw_text.split('\n') if line.strip()]
        grade_val = None
        justification_val = None
        for line in lines:
            low = line.lower()
            if low.startswith('grade:') and grade_val is None:
                possible = line.split(':', 1)[1].strip().upper()
                if possible and possible[0] in {'A','B','C','D','F'}:
                    grade_val = possible[0]
            elif low.startswith('justification:') and justification_val is None:
                justification_val = line.split(':', 1)[1].strip()

        # Fallback extraction if not properly structured.
        if grade_val is None:
            # Search for standalone letter
            for cand in ['A','B','C','D','F']:
                if f' {cand} ' in f' {raw_text} ':
                    grade_val = cand
                    break
        if grade_val is None:
            grade_val = 'F'  # default fail-safe
        if not justification_val:
            justification_val = 'Answer lacks sufficient alignment with the provided summary.'

        # Truncate overly long justification
        if len(justification_val) > 280:
            justification_val = justification_val[:277] + '...'

        state.grading = f"Grade: {grade_val}\nJustification: {justification_val}"
        preview = (state.grading[:100] + "...") if len(state.grading) > 100 else state.grading
        state.messages.append({
            "role": "assistant", "content": f"Completed grading for {state.topic}",
            "action": "grade_quiz_complete", "grading_preview": preview
        })
        return state

    def present_feedback(self, state: HealthBotState) -> HealthBotState:
        print("\nYour grade and feedback:\n")
        grading_text = state.grading or ""
        lines = grading_text.strip().split('\n')
        grade_line = ""
        justification_line = ""
        for ln in lines:
            lower = ln.strip().lower()
            if lower.startswith('grade:'):
                grade_line = ln.strip()
            elif lower.startswith('justification:'):
                justification_line = ln.strip()
            elif grade_line and not justification_line and ln.strip():
                justification_line = "Justification: " + ln.strip()
        if grade_line:
            print(grade_line)
        if justification_line:
            print(justification_line)
        if not grade_line or not justification_line:
            print(grading_text)
        # Ask user for next action to set continue_flag
        try:
            choice = input("\nWhat next? (quiz=another quiz question, new=new topic, enter=exit): ").strip().lower()
        except EOFError:
            choice = ""
        if choice == 'quiz':
            state.continue_flag = 'quiz'
        elif choice == 'new':
            state.continue_flag = 'new'
        else:
            state.continue_flag = None
        state.messages.append({
            "role": "user",
            "content": f"Next action choice: {choice or 'exit'}",
            "action": "post_feedback_choice"
        })
        return state

In [4]:
# /usr/bin/env python3
"""
healthAiBot graph definition.
"""

from datetime import datetime
import json
from langgraph.graph import StateGraph, END
from langgraph.prebuilt import ToolNode


try:
    from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ToolMessage, BaseMessage
except ImportError:  # Fallback if package structure differs
    from langchain.schema import SystemMessage, HumanMessage, AIMessage, BaseMessage  # type: ignore
    try:
        from langchain.schema import ToolMessage  # type: ignore
    except ImportError:  # Define minimal ToolMessage
        class ToolMessage(HumanMessage):  # type: ignore
            def __init__(self, content: str, name: str, tool_call_id: str = ""):
                super().__init__(content=content)
                self.name = name
                self.tool_call_id = tool_call_id


# Feedback router for conditional graph edges after present_feedback
def feedback_router(state: HealthBotState):
    if state.continue_flag == 'quiz':
        return "create_quiz"
    elif state.continue_flag == 'new':
        return "reset_topic_state"
    else:
        return END

def build_healthbot_graph(model) -> StateGraph:
    """
    Build the HealthBot graph with nodes and transitions, using HealthBotState and ToolNode for Tavily search.
    """
    helper = GraphHelper()
    graph = StateGraph(HealthBotState)

    # Real ToolNode usage with tavily_search_tool defined as a LangChain tool.
    tool_node = ToolNode([tavily_search_tool])

    def ensure_tool_call(state: HealthBotState) -> HealthBotState:
        """Ensure there is an AIMessage with a tool call for tavily_search_tool.

        Converts legacy dict messages to LangChain message objects if needed so ToolNode can parse them.
        """
        if not state.messages:
            return state

        # If messages are dicts, convert them.
        if not isinstance(state.messages[0], BaseMessage):
            converted = []
            for m in state.messages:
                role = m.get("role")
                content = m.get("content", "")
                if role == "system":
                    converted.append(SystemMessage(content=content))
                elif role == "user":
                    converted.append(HumanMessage(content=content))
                elif role == "assistant":
                    tool_calls = m.get("tool_calls") or []
                    lc_tool_calls = []
                    for tc in tool_calls:
                        args = tc.get("arguments") or tc.get("args") or {}
                        if isinstance(args, str):
                            try:
                                args = json.loads(args)
                            except Exception:
                                args = {"raw": args}
                        lc_tool_calls.append({
                            "id": tc.get("id"),
                            "name": tc.get("name"),
                            "args": args,
                        })
                    converted.append(AIMessage(content=content, tool_calls=lc_tool_calls))
                elif role == "tool":
                    converted.append(ToolMessage(content=content, name=m.get("name", "tool"), tool_call_id=m.get("id") or m.get("tool_call_id", "")))
                else:
                    converted.append(HumanMessage(content=content))
            state.messages = converted

        # Now operate on LangChain messages
        last = state.messages[-1]
        if isinstance(last, AIMessage):
            if not getattr(last, 'tool_calls', None):
                last.tool_calls = [{
                    "id": "auto_tavily_" + str(datetime.now().timestamp()),
                    "name": "tavily_search_tool",
                    "args": {"topic": state.topic}
                }]
        else:
            # Append a new AIMessage with tool call
            state.messages.append(AIMessage(content=f"Initiating search for {state.topic}", tool_calls=[{
                "id": "auto_tavily_" + str(datetime.now().timestamp()),
                "name": "tavily_search_tool",
                "args": {"topic": state.topic}
            }]))
        return state

    def process_tool_output(state: HealthBotState) -> HealthBotState:
        """Extract last tool message content into state.search_results for downstream summarization."""
        # Search from end for a ToolMessage (LangChain) first
        found = False
        for msg in reversed(state.messages):
            if isinstance(msg, ToolMessage):
                state.search_results = msg.content
                found = True
                break
            # Legacy dict form
            if isinstance(msg, dict) and msg.get("role") == "tool":
                state.search_results = msg.get("content", "")
                found = True
                break
        if not found or not state.search_results:
            # Fallback: invoke tool directly
            try:
                import os
                if not os.environ.get("TAVILY_API_KEY"):
                    state.search_results = "Missing Tavily API key. Please export TAVILY_API_KEY to enable search."
                else:
                    # Prefer direct call; tavily_search_tool supports .invoke when decorated
                    if hasattr(tavily_search_tool, 'invoke'):
                        fallback_result = tavily_search_tool.invoke({"topic": state.topic})
                    else:
                        fallback_result = tavily_search_tool(state.topic or "")
                    state.search_results = str(fallback_result)
                    # Record as ToolMessage for consistency
                    state.messages.append(ToolMessage(content=f"(Fallback) Search completed for {state.topic}.", name="tavily_search_tool", tool_call_id="fallback"))
            except Exception as e:
                state.search_results = f"No search results captured and fallback failed: {e}"
        return state

    def reset_topic_state(state: HealthBotState) -> HealthBotState:
        """Clear topic-specific fields before starting a new topic cycle."""
        state.focus = None
        state.search_results = None
        state.summary = None
        state.quiz_question = None
        state.quiz_answer = None
        state.grading = None
        state.previous_questions = []
        state.continue_flag = None
        # Do not clear messages entirely to retain audit trail; append a separator marker
        try:
            from langchain_core.messages import HumanMessage  # type: ignore
            state.messages.append(HumanMessage(content="--- NEW TOPIC ---"))
        except Exception:
            state.messages.append({"role": "user", "content": "--- NEW TOPIC ---"})
        return state

    # Add all nodes to the graph
    # Core information gathering & presentation nodes
    graph.add_node("ask_patient", helper.ask_patient)
    graph.add_node("generate_assistant_message", helper.generate_assistant_message)
    graph.add_node("ensure_tool_call", ensure_tool_call)
    graph.add_node("search_tavily", tool_node)
    graph.add_node("process_tool_output", process_tool_output)
    graph.add_node("ask_for_focus", helper.ask_for_focus)
    graph.add_node("summarize_results", helper.summarize_results)
    graph.add_node("present_summary", helper.present_summary)
    graph.add_node("comprehension_prompt", helper.comprehension_prompt)

    # Quiz / feedback flow nodes (previously unwired)
    graph.add_node("create_quiz", helper.create_quiz)
    graph.add_node("present_quiz", helper.present_quiz)
    graph.add_node("get_quiz_answer", helper.get_quiz_answer)
    graph.add_node("grade_quiz", helper.grade_quiz)
    graph.add_node("present_feedback", helper.present_feedback)
    graph.add_node("reset_topic_state", reset_topic_state)
    graph.add_edge("ask_patient", "generate_assistant_message")
    graph.add_edge("generate_assistant_message", "ensure_tool_call")
    graph.add_edge("ensure_tool_call", "search_tavily")
    graph.add_edge("search_tavily", "process_tool_output")
    graph.add_edge("process_tool_output", "ask_for_focus")
    graph.add_edge("ask_for_focus", "summarize_results")
    graph.add_edge("summarize_results", "present_summary")
    graph.add_edge("present_summary", "comprehension_prompt")

    # After comprehension prompt we always generate one quiz for now.
    # If future logic sets continue_flag we can branch using a conditional edge.
    graph.add_edge("comprehension_prompt", "create_quiz")
    graph.add_edge("create_quiz", "present_quiz")
    graph.add_edge("present_quiz", "get_quiz_answer")
    graph.add_edge("get_quiz_answer", "grade_quiz")
    graph.add_edge("grade_quiz", "present_feedback")
    # Conditional routing after feedback: END (default) | quiz (new question) | new (restart)
    graph.add_conditional_edges(
        "present_feedback",
        feedback_router,
        {
            "create_quiz": "create_quiz",  # repeat quiz with new question
            "reset_topic_state": "reset_topic_state",  # reset then new topic
            END: END,
        },
    )
    # After resetting topic-specific state, return to ask_patient for a fresh cycle
    graph.add_edge("reset_topic_state", "ask_patient")
    graph.set_entry_point("ask_patient")

    return graph


In [5]:
# /usr/bin/env python3
"""
HealthBot Command Line Interface (CLI)
This script provides a command-line interface for interacting with the HealthBot application.
Users can specify various parameters such as the LLM backend, model name, and temperature.
"""

#import argparse


def main():
    """
    Main function to run the HealthBot CLI.
    """

    llm_type = "ollama"
    model_name = "gemma3:1b"
    temperature = 0.3

    healthbot = HealthBotUtils(
        llm_type=llm_type,
        model_name=model_name,
        temperature=temperature,
    )
    
    llm = healthbot.get_llm()


    graph = build_healthbot_graph(llm)
    app = graph.compile()

    print("Welcome to HealthBot!")
    # Single invocation; looping & quiz handled internally by graph via continue_flag routing
    state = healthbot.reset_state(llm)
    state = app.invoke(state, config={"recursion_limit": 100})
    # If user chose to start a new topic or additional quizzes, the graph's conditional edges manage it;
    # CLI exits after first completed flow.
    print("\nThank you for using HealthBot. Stay healthy!")

In [6]:
main()

Welcome to HealthBot!


What health topic or medical condition would you like to learn about?  common cold


You have chosen to learn about: common cold


Do you want to focus on a specific aspect (e.g., symptoms, treatment, prevention)? If yes, enter it, otherwise press Enter:  symptoms



Here is a summary of what you asked about:

The search results indicate that the common cold is a frequently discussed illness, with several resources providing information about its symptoms, potential causes, and management. The Wikipedia page offers a basic overview of the disease, including its viral nature and common symptoms like runny nose, nasal congestion, and sneezing. The CDC website details the typical symptoms and offers recommendations for managing cold symptoms, including over-the-counter medications and probiotics. The MedlinePlus encyclopedia provides a more detailed explanation of the disease, including its potential causes and diagnostic considerations, while also including information about the most common symptoms and remedies. The PMC article delves deeper into the underlying mechanisms of the common cold, including its relationship to allergies and asthma, and offers insights into potential preventative measures.

The search results provide a range of perspectiv

...


Quiz Question:

What is the primary focus of the search results regarding the common cold?



Enter your answer to the quiz question:  symptoms



Your grade and feedback:

Grade: B
Justification: The summary describes the search results as focusing on the disease itself, including its symptoms, potential causes, and management strategies.



What next? (quiz=another quiz question, new=new topic, enter=exit):  exit



Thank you for using HealthBot. Stay healthy!
